In [117]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor


def carregar_dados(file_name):
    # Carregar dados
    dados = pd.read_csv(file_name)

    # Normalizar colunas com vírgulas e porcentagens
    for coluna in dados.columns:
        if dados[coluna].dtype == object:
            if '%' in dados[coluna].iloc[0]:
                dados[coluna] = dados[coluna].replace('%', '', regex=True).str.replace(',', '.', regex=True).astype(float) / 100.0
            elif ',' in dados[coluna].iloc[0]:
                dados[coluna] = dados[coluna].str.replace(',', '.', regex=True).astype(float)

    return dados


def normalizar_dados(dados, colunas_preditoras):
    # Excluir colunas não numéricas das colunas_preditoras
    colunas_numericas = [coluna for coluna in colunas_preditoras if pd.api.types.is_numeric_dtype(dados[coluna])]
    if 'TIME' in dados.columns:
        dados = dados.drop('TIME', axis=1)


    # Normalizar apenas as colunas numéricas
    scaler = StandardScaler()
    dados[colunas_numericas] = scaler.fit_transform(dados[colunas_numericas])

    return dados

def dividir_dados(dados, coluna_alvo):
    X = dados.drop(coluna_alvo, axis=1)
    y = dados[coluna_alvo]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def treinar_modelo(dados, colunas_preditoras, coluna_alvo):
    # Separar dados em conjunto de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(dados[colunas_preditoras], dados[coluna_alvo], test_size=0.2, random_state=42)

    # Inicializar o modelo
    modelo = RandomForestRegressor(random_state=42)

    # Treinar o modelo
    modelo.fit(X_train, y_train)

    # Avaliar o modelo no conjunto de teste
    avaliar_modelo(modelo, X_test, y_test)

    return modelo
    
def avaliar_modelo(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Squared Error no conjunto de teste: {mse:.2f}")
    print(f"Mean Absolute Error no conjunto de teste: {mae:.2f}")
    print(f"R-squared no conjunto de teste: {r2:.2f}")

    return mse, mae, r2


def prever_probabilidade(modelo, dados, nome_time_1, nome_time_2, colunas):
    # Obter dados dos times
    dados_time_1 = dados[dados['TIME'] == nome_time_1][colunas]
    dados_time_2 = dados[dados['TIME'] == nome_time_2][colunas]

    # Criar DataFrame com dados dos times
    dados_predicao = pd.DataFrame({
        'PPG': [dados_time_1['PPG'].values[0], dados_time_2['PPG'].values[0]],
        'CS': [dados_time_1['CS'].values[0], dados_time_2['CS'].values[0]],
        # Adicione outras colunas conforme necessário
    })

    # Prever probabilidades
    probabilidades = modelo.predict(dados_predicao)

    # Calcular probabilidade de vitória e empate
    probabilidade_vitoria_time_1 = probabilidades[0]
    probabilidade_vitoria_time_2 = probabilidades[1]
    probabilidade_empate = 1 - (probabilidade_vitoria_time_1 + probabilidade_vitoria_time_2)

    return probabilidade_vitoria_time_1, probabilidade_empate, probabilidade_vitoria_time_2





In [118]:

# Exemplo de uso
file_name = '../dados/combinado.csv'  # Substitua pelo caminho real do seu arquivo
coluna_alvo = 'Pts'

In [119]:


# Carregar dados
dados = carregar_dados(file_name)

# Normalizar dados

In [120]:
# Normalizar dados
colunas_preditoras = ['FINALIZAÇÕES', 'GOLS', 'MÉDIA DE CHUTES P/ JOGO', 'APROVEITAMENTO', 'GP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PPG', 'last 8', 'CS', 'FTS', 'TC', 'MC']
coluna_alvo = 'Pts'  # Substitua pelo nome da sua coluna alvo

dados_normalized = normalizar_dados(dados, colunas_preditoras)


In [121]:


# Dividir dados
X_train, X_test, y_train, y_test = dividir_dados(dados_normalized, coluna_alvo)



In [122]:


# Treinar modelo
modelo = treinar_modelo(dados, colunas_preditoras, coluna_alvo)



# Avaliar modelo
avaliar_modelo(modelo, X_test, y_test)



Mean Squared Error no conjunto de teste: 13.91
Mean Absolute Error no conjunto de teste: 3.14
R-squared no conjunto de teste: 0.88
Mean Squared Error no conjunto de teste: 159.06
Mean Absolute Error no conjunto de teste: 10.73
R-squared no conjunto de teste: -0.34


(159.06005000000002, 10.73, -0.34227890295358665)

In [123]:


# Nomes fictícios de dois times (substitua pelos nomes reais)
time_1 ='Palmeiras'
time_2 = 'Coritiba'
nome_time_1 = f'\xa0{time_1}'
nome_time_2 = f'\xa0{time_2}'
# Nomes fictícios de dois times (substitua pelos nomes reais)

# Prever probabilidades
prob_vitoria_1, prob_empate, prob_vitoria_2 = prever_probabilidade(modelo, dados, time_1, time_2, colunas_preditoras)

print(f'Probabilidade de Vitória para {time_1}: {prob_vitoria_1:.2%}')
print(f'Probabilidade de Empate: {prob_empate:.2%}')
print(f'Probabilidade de Vitória para {time_2}: {prob_vitoria_2:.2%}')

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- APROVEITAMENTO
- D
- FINALIZAÇÕES
- FTS
- GA
- ...
